In [3]:
import gzip, json, torch
import torch.nn as nn
from torch.utils.data import Dataset
from rdkit import Chem
from rdkit.Chem import AllChem
# -- Morgan‑FP helpers (unchanged) -----------------------------------------
GEN0 = AllChem.GetMorganGenerator(radius=0, fpSize=2048)
GEN1 = AllChem.GetMorganGenerator(radius=1, fpSize=2048)
GEN2 = AllChem.GetMorganGenerator(radius=2, fpSize=2048)

def fp6144_from_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp0 = torch.tensor(list(GEN0.GetFingerprint(mol)), dtype=torch.float32)
    fp1 = torch.tensor(list(GEN1.GetFingerprint(mol)), dtype=torch.float32)
    fp2 = torch.tensor(list(GEN2.GetFingerprint(mol)), dtype=torch.float32)
    return torch.cat([fp0, fp1, fp2])        # shape (6144,)

# -- NEW dataset class -----------------------------------------------------
class JSONLPotencyDataset(Dataset):
    """
    Loads the new *.jsonl.gz split files created in the parsing notebook.
    Each line contains {"smiles": ..., "label_vector": [...] }.
    """
    def __init__(self, path_to_jsonl_gz):
        with gzip.open(path_to_jsonl_gz, "rt") as f:
            self.records = [(rec["smiles"], rec["label_vector"])
                            for rec in (json.loads(l) for l in f)]

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        smiles, label_vec = self.records[idx]
        fp = fp6144_from_smiles(smiles)

        # fall‑through if RDKit fails on this SMILES
        while fp is None:
            idx = (idx + 1) % len(self.records)
            smiles, label_vec = self.records[idx]
            fp = fp6144_from_smiles(smiles)

        labels = torch.tensor(label_vec, dtype=torch.long)   # (60,)
        return fp, labels

In [4]:
class MultiLineMLP5(nn.Module):
    def __init__(self,
                 input_dim=6144,
                 hidden_dims=[1024, 1024, 512, 512, 256],
                 num_lines=60,
                 num_classes=6,
                 p_drop=0.3):
        super().__init__()

        self.bn_in = nn.BatchNorm1d(input_dim)

        layers = []
        dims = [input_dim] + hidden_dims
        for d_in, d_out in zip(dims[:-1], dims[1:]):
            layers.extend([
                nn.Linear(d_in, d_out),
                nn.BatchNorm1d(d_out),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ])
        self.shared = nn.Sequential(*layers)

        self.classifier = nn.Linear(hidden_dims[-1], num_lines * num_classes)

    def forward(self, x):
        x = self.bn_in(x)
        x = self.shared(x)
        logits = self.classifier(x)
        return logits.view(-1, 60, 6)



In [5]:
import torch, json, numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
from rdkit import RDLogger

# Silence RDKit chatter
RDLogger.DisableLog("rdApp.*")

# ---------- DATA ----------------------------------------------------------
train_dataset = JSONLPotencyDataset("train.jsonl.gz")
val_dataset   = JSONLPotencyDataset("val.jsonl.gz")

train_loader  = DataLoader(train_dataset, batch_size=64,
                           shuffle=True, num_workers=4, pin_memory=True)
val_loader    = DataLoader(val_dataset,   batch_size=64,
                           num_workers=4, pin_memory=True)

# ---------- MODEL ---------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MultiLineMLP5().to(device)


In [6]:
from tqdm.auto import tqdm
import torch.nn.functional as F

# ---------- 1.  class‑imbalance weights  ----------------------------------
hist = torch.zeros(6)
for _, labels in train_loader:
    mask = labels != -1
    for c in range(6):
        hist[c] += ((labels == c) & mask).sum()

weights = 1.0 / (hist + 1e-6)
weights = (weights / weights.sum()) * 6
weights = weights.to(torch.float32).to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, weight=weights)

# ---------- 2.  full‑set validation accuracy ------------------------------
def full_val_accuracy(model, loader, device):
    correct = np.zeros(60); total = np.zeros(60)
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            p    = model(x).argmax(2)
            m    = y != -1
            correct += ((p == y) & m).sum(0).cpu().numpy()
            total   += m.sum(0).cpu().numpy()
    accs = [c / t if t > 0 else None for c, t in zip(correct, total)]
    return float(np.nanmean(accs)), accs

# ---------- 3.  optimiser, scheduler, loop --------------------------------
optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2)

best_acc = 0.0
num_epochs = 40
for epoch in range(1, num_epochs + 1):
    # TRAIN
    model.train()
    epoch_loss = 0.0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss   = criterion(logits.view(-1, 6), y.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        epoch_loss += loss.item()

    # VALIDATE
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            val_loss += criterion(
                model(x).view(-1, 6), y.view(-1)
            ).item()

    scheduler.step(val_loss)
    avg_acc, _ = full_val_accuracy(model, val_loader, device)
    print(f"Epoch {epoch:02}/{num_epochs} | "
          f"Train Loss: {epoch_loss:.1f} | Val Loss: {val_loss:.1f} | "
          f"Avg Val Acc: {avg_acc:.4f} | "
          f"LR: {optimizer.param_groups[0]['lr']:.1e}")

    if avg_acc > best_acc:
        best_acc = avg_acc
        torch.save(model.state_dict(), "best_model.pt")
        print(f"  ✓ saved new best ({best_acc:.4f})")

print(f"Done. Best Avg Val Acc = {best_acc:.4f}")


/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                                                                                           

Epoch 01/40 | Train Loss: 1082.8 | Val Loss: 158.7 | Avg Val Acc: 0.3560 | LR: 5.0e-04
  ✓ saved new best (0.3560)


Epoch 02/40 | Train Loss: 913.2 | Val Loss: 171.2 | Avg Val Acc: 0.3709 | LR: 5.0e-04
  ✓ saved new best (0.3709)


Epoch 03/40 | Train Loss: 844.2 | Val Loss: 184.7 | Avg Val Acc: 0.3666 | LR: 5.0e-04


Epoch 04/40 | Train Loss: 806.9 | Val Loss: 173.7 | Avg Val Acc: 0.3578 | LR: 2.5e-04


...

Epoch 30/40 | Train Loss: 512.9 | Val Loss: 196.4 | Avg Val Acc: 0.4184 | LR: 9.8e-07


Epoch 31/40 | Train Loss: 511.9 | Val Loss: 206.3 | Avg Val Acc: 0.4170 | LR: 4.9e-07


Epoch 32/40 | Train Loss: 511.3 | Val Loss: 189.8 | Avg Val Acc: 0.4167 | LR: 4.9e-07


Exception in thread Thread-136:██████████████████▋       | 603/730 [00:42<00:07, 17.77it/s]
Traceback (most recent call last):
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/nbilic/miniconda3/envs/Nandos/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _F

KeyboardInterrupt: 